First I'll try to not use Selenium and see if scraping is possible simply through using a request.

In [1]:
import requests

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
}

In [3]:
res = requests.get("https://twitter.com/GTNUK1", headers=headers, timeout=5)

In [4]:
res.status_code

200

After some scavenging I've found out the CSS selectors used by Twitter for the relevant data (i.e. Bio, Following Count, Followers Count, Location, Website).

In [35]:
CONTAINER = "[data-testid='primaryColumn']"
selectors = {
    "Bio": "[data-testid='UserDescription']",
    "FollowingFollowers": ".css-175oi2r:not(.r-1mf7evn) .r-bcqeeo.r-qvutc0.r-poiln3.r-1b43r93.r-1cwl3u0.r-b88u0q span",
    "Location": "[data-testid='UserLocation']",
    "Website": "[data-testid='UserUrl']",
}

In [2]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(res.text, "html.parser")

NameError: name 'res' is not defined

In [8]:
information = []

for key in selectors:
    print(soup.select(selectors[key]))

[]
[]
[]
[]
[]


Clearly this doesn't work...  
Workaround: Use selenium instead.

In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [40]:
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)

Problem reading geckodriver versions: error sending request for url (https://raw.githubusercontent.com/SeleniumHQ/selenium/trunk/common/geckodriver/geckodriver-support.json): error trying to connect: tcp connect error: Operation timed out (os error 110). Using latest geckodriver version


In [20]:
driver.get("https://twitter.com/IndiaWeatherMan")
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, selectors["Followers"]))
)
container = driver.find_element(By.CSS_SELECTOR, CONTAINER).get_attribute("innerHTML")
driver.quit()

In [21]:
soup = BeautifulSoup(container, "html.parser")

In [36]:
bio = soup.select_one(selectors["Bio"])

following_followers = soup.select(selectors["FollowingFollowers"])
following = following_followers[0]
followers = following_followers[1]

location = soup.select_one(selectors["Location"])
website = soup.select_one(selectors["Website"])

In [37]:
following_followers

[<span class="css-1qaijid r-bcqeeo r-qvutc0 r-poiln3" style="text-overflow: unset;">0</span>,
 <span class="css-1qaijid r-bcqeeo r-qvutc0 r-poiln3" style="text-overflow: unset;">139.9K</span>]

In [38]:
(
    bio.text if bio else None,
    following.text,
    followers.text,
    location.text if location else None,
    website.text if website else None,
)

('No Affiliation To Any Official Agency. Not Pro, Just Weather Enthusiast.',
 '0',
 '139.9K',
 None,
 None)